In [1]:
!pip install SimpleITK

import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import re
import shutil
import random

In [2]:
def load_all_images(file):
    print(f'loading the {file} file...')
    
    ### LOAD SEGMENTATION FILES
    seg1_image = sitk.ReadImage(f'data/{file}/ManualSegmentation_1.nii.gz')  
    seg2_image = sitk.ReadImage(f'data/{file}/ManualSegmentation_2.nii.gz')  
    seg3_image = sitk.ReadImage(f'data/{file}/ManualSegmentation_3.nii.gz')  
    seg4_image = sitk.ReadImage(f'data/{file}/ManualSegmentation_4.nii.gz')  
    seg5_image = sitk.ReadImage(f'data/{file}/ManualSegmentation_5.nii.gz') 
    seg6_image = sitk.ReadImage(f'data/{file}/ManualSegmentation_6.nii.gz') 
    seg7_image = sitk.ReadImage(f'data/{file}/ManualSegmentation_7.nii.gz')

    ### LOAD 3D SCAN FILE
    flair_image = sitk.ReadImage(f'data/{file}/3DFLAIR.nii.gz')

    size = seg1_image.GetSize()

    caster = sitk.CastImageFilter()
    caster.SetOutputPixelType(sitk.sitkUInt8)

    ### FIND THE SLICES IN Z AXIS
    for z_slice in range(size[0]):
        # iterated slice of 3d image
        sliced_images = [
            seg1_image[z_slice, :, :],
            seg2_image[z_slice, :, :],
            seg3_image[z_slice, :, :],
            seg4_image[z_slice, :, :],
            seg5_image[z_slice, :, :],
            seg6_image[z_slice, :, :],
            seg7_image[z_slice, :, :]
        ]

        # get array from image slice
        sliced_arrays = [
            sitk.GetArrayFromImage(sliced_images[0]),
            sitk.GetArrayFromImage(sliced_images[1]),
            sitk.GetArrayFromImage(sliced_images[2]),
            sitk.GetArrayFromImage(sliced_images[3]),
            sitk.GetArrayFromImage(sliced_images[4]),
            sitk.GetArrayFromImage(sliced_images[5]),
            sitk.GetArrayFromImage(sliced_images[6])
        ]

        # same for the 3d image
        flair_slice_image = flair_image[z_slice, :, :]
        #flair_slice_array = sitk.GetArrayFromImage(flair_slice_image)

        for count, slice_array in enumerate(sliced_arrays):
            # if there are only zero values, skip
            non_zero_values = slice_array[slice_array != 0]
            if non_zero_values.size == 0:
                continue

            binary_mask = (slice_array > 0).astype(np.uint8)
            output_image = sitk.GetImageFromArray(binary_mask)

            # add origin, spacing, direction from original image
            output_image.SetOrigin(sliced_images[count].GetOrigin())
            output_image.SetSpacing(sliced_images[count].GetSpacing())
            output_image.SetDirection(sliced_images[count].GetDirection())

            # output image
            png_output_path = f'data/train/labels/{file}_seg{count+1}_{z_slice:03}.png'
            sitk.WriteImage(output_image, png_output_path)

            #output for the 3d image
            flair_slice_image = sitk.RescaleIntensity(flair_slice_image, 0, 255)
            flair_slice_image = caster.Execute(flair_slice_image)
            #output_image = sitk.GetImageFromArray(flair_slice_array.astype(np.uint8))
            #output_image.SetOrigin(flair_slice_image.GetOrigin())
            #output_image.SetSpacing(flair_slice_image.GetSpacing())
            #output_image.SetDirection(flair_slice_image.GetDirection())
            #png_output_path = f'data/train/images/{file}_{z_slice:03}.png'
            #sitk.WriteImage(flair_slice_image, png_output_path)
            #png_output_path = f'data/train/images_all/{file}_seg{count+1}_{z_slice:03}.png'
            #sitk.WriteImage(flair_slice_image, png_output_path)
            png_output_path = f'data/train/images/{file}_seg{count+1}_{z_slice:03}.png'
            sitk.WriteImage(flair_slice_image, png_output_path)

    print(f'...finished reading outputing the {file} file')

In [3]:
# loading images onto drive
load_all_images('01016SACH')
load_all_images('01040VANE')
load_all_images('07010NABO')
load_all_images('07040DORE')
load_all_images('07043SEME')
load_all_images('08002CHJE')

loading the 01016SACH file...
...finished reading outputing the 01016SACH file
loading the 01040VANE file...
...finished reading outputing the 01040VANE file
loading the 07010NABO file...
...finished reading outputing the 07010NABO file
loading the 07040DORE file...
...finished reading outputing the 07040DORE file
loading the 07043SEME file...
...finished reading outputing the 07043SEME file
loading the 08002CHJE file...
...finished reading outputing the 08002CHJE file


In [4]:
def move_related_files(source_dir1, source_dir2, dest_dir1, dest_dir2, percentage):
    files1 = os.listdir(source_dir1)
    files2 = os.listdir(source_dir2)

    os.makedirs(dest_dir1, exist_ok=True)
    os.makedirs(dest_dir2, exist_ok=True)

    num_files_to_move = int(min(len(files1), len(files2)) * (percentage / 100.0))

    files_to_move = random.sample(list(zip(files1, files2)), num_files_to_move)
    #print(files_to_move)

    for file_pair in files_to_move:
        file1, file2 = file_pair
        #print(f"Moving: {file1} and {file2}")
        source_path1 = os.path.join(source_dir1, file1)
        source_path2 = os.path.join(source_dir2, file2)
        dest_path1 = os.path.join(dest_dir1, file1)
        dest_path2 = os.path.join(dest_dir2, file2)

        try:
            shutil.move(source_path1, dest_path1)
            shutil.move(source_path2, dest_path2)
        except Exception as e:
            print(e)

        print(f"Moved: {file1} and {file2}")
        

In [5]:
# move some images and labels to test set
source_directory1 = "data/train/images"
source_directory2 = "data/train/labels"
destination_directory1 = "data/test/images"
destination_directory2 = "data/test/labels"
percentage_to_move = 20  # Adjust this percentage as needed
move_related_files(source_directory1, source_directory2, destination_directory1, destination_directory2, percentage_to_move)


Moved: 07043SEME_seg4_095.png and 07043SEME_seg4_095.png
Moved: 01016SACH_seg5_124.png and 01016SACH_seg5_124.png
Moved: 08002CHJE_seg5_065.png and 08002CHJE_seg5_065.png
Moved: 07040DORE_seg2_163.png and 07040DORE_seg2_163.png
Moved: 01016SACH_seg7_074.png and 01016SACH_seg7_074.png
Moved: 08002CHJE_seg7_064.png and 08002CHJE_seg7_064.png
Moved: 07040DORE_seg2_112.png and 07040DORE_seg2_112.png
Moved: 07043SEME_seg2_075.png and 07043SEME_seg2_075.png
Moved: 07010NABO_seg2_169.png and 07010NABO_seg2_169.png
Moved: 08002CHJE_seg4_104.png and 08002CHJE_seg4_104.png
Moved: 08002CHJE_seg4_216.png and 08002CHJE_seg4_216.png
Moved: 08002CHJE_seg2_138.png and 08002CHJE_seg2_138.png
Moved: 07010NABO_seg6_124.png and 07010NABO_seg6_124.png
Moved: 07040DORE_seg7_110.png and 07040DORE_seg7_110.png
Moved: 07043SEME_seg5_085.png and 07043SEME_seg5_085.png
Moved: 08002CHJE_seg6_206.png and 08002CHJE_seg6_206.png
Moved: 08002CHJE_seg6_146.png and 08002CHJE_seg6_146.png
Moved: 01040VANE_seg4_108.png a

In [6]:
def generate_consensus_labels():
    image_path = 'data/test/images'
    label_path = 'data/test/labels'
    
    caster = sitk.CastImageFilter()
    caster.SetOutputPixelType(sitk.sitkUInt8)

    # Iterate over all files in the folder
    for filename in os.listdir(image_path):
        file_path = os.path.join(image_path, filename)

        # Check if it's a file (not a subdirectory)
        if os.path.isfile(file_path):
            # Extract information using regex
            parts = filename.split('_')

            if len(parts) == 3:
                # Extract imageName and slice number
                imageName, fileType, slice_extension = parts
                sliceNumber, extension = slice_extension.split('.')
                sliceNumber = int(sliceNumber.lstrip('0'))
                
                print(f"Generating consensus for: {imageName}-{sliceNumber}...")
                
                ### LOAD SEGMENTATION FILES
                con_image = sitk.ReadImage(f'data/{imageName}/Consensus.nii.gz')
                
                sliced_image = con_image[sliceNumber, :, :]
                sliced_array = sitk.GetArrayFromImage(sliced_image)
                
                binary_mask = (sliced_array > 0).astype(np.uint8)
                output_image = sitk.GetImageFromArray(binary_mask)
                
                output_image.SetOrigin(sliced_image.GetOrigin())
                output_image.SetSpacing(sliced_image.GetSpacing())
                output_image.SetDirection(sliced_image.GetDirection())
                
                sliceNumber = str(sliceNumber).zfill(3)
                png_output_path = f'{label_path}/{imageName}_{sliceNumber}.png'
                sitk.WriteImage(output_image, png_output_path)
                
                file_path_to_delete = f'{label_path}/{imageName}_{fileType}_{sliceNumber}.{extension}'
                if os.path.exists(file_path_to_delete):
                    os.remove(file_path_to_delete)

                # Create the new file name without {type}
                new_filename = f'{imageName}_{sliceNumber}.{extension}'  # Adjust the file extension if needed
                if not os.path.exists(os.path.join(image_path, new_filename)):
                    os.rename(file_path, os.path.join(image_path, new_filename))
                    
                file_path_to_delete = f'{image_path}/{imageName}_{fileType}_{sliceNumber}.{extension}'
                if os.path.exists(file_path_to_delete):
                    os.remove(file_path_to_delete)


In [7]:
# generate consensus images for test set
generate_consensus_labels()


Generating consensus for: 01016SACH-54...
Generating consensus for: 01016SACH-64...
Generating consensus for: 01016SACH-76...
Generating consensus for: 01016SACH-95...
Generating consensus for: 01016SACH-97...
Generating consensus for: 01016SACH-98...
Generating consensus for: 01016SACH-106...
Generating consensus for: 01016SACH-126...
Generating consensus for: 01016SACH-127...
Generating consensus for: 01016SACH-130...
Generating consensus for: 01016SACH-132...
Generating consensus for: 01016SACH-134...
Generating consensus for: 01016SACH-136...
Generating consensus for: 01016SACH-141...
Generating consensus for: 01016SACH-144...
Generating consensus for: 01016SACH-145...
Generating consensus for: 01016SACH-150...
Generating consensus for: 01016SACH-160...
Generating consensus for: 01016SACH-161...
Generating consensus for: 01016SACH-166...
Generating consensus for: 01016SACH-174...
Generating consensus for: 01016SACH-176...
Generating consensus for: 01016SACH-178...
Generating consen